In [1]:
import pandas as pd
import numpy as np

In [138]:
train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')
submission = pd.read_csv('submission.csv')

In [140]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 24414 entries, 0 to 24413
Data columns (total 6 columns):
Id                24414 non-null int64
Country_Region    24414 non-null object
Province_State    10374 non-null object
Date              24414 non-null object
ConfirmedCases    24414 non-null float64
Fatalities        24414 non-null float64
dtypes: float64(2), int64(1), object(3)
memory usage: 1.1+ MB


In [51]:
for data in [train,test]:
    data['Date'] = data['Date'].apply(lambda date : pd.to_datetime(date))
    data['month'] = data['Date'].apply(lambda date: date.month)
    data['day'] = data['Date'].apply(lambda date: date.day)
    day_from_jan_first = data['Date'] - pd.to_datetime('2020-01-01')
    data['day_from_jan_first'] = day_from_jan_first.apply(lambda day_obj : day_obj.days + 1)

In [52]:
train['Country/Province'] = np.where(train['Province_State'].isna() == False, train['Country_Region'] + '/' + train['Province_State'], train['Country_Region'])
test['Country/Province'] = np.where(test['Province_State'].isna() == False, test['Country_Region'] + '/' + test['Province_State'], test['Country_Region'])

In [54]:
countries = train['Country/Province'].unique()

In [128]:
from xgboost import XGBRegressor
submission=pd.DataFrame(columns=submission.columns)
for country in countries:
    train_df=train[train['Country/Province']==country]
    x=train_df[['day_from_jan_first','month', 'day']]
    x_fat = train_df[['day_from_jan_first','month', 'day','ConfirmedCases']]
    y1=train_df[['ConfirmedCases']]
    y2=train_df[['Fatalities']]
    model_1=XGBRegressor()
    model_2=XGBRegressor()
    model_1.fit(x,y1)
    model_2.fit(x_fat,y2)
    test_df=test[test['Country/Province']==country]
    test_id=test_df['Id'].values.tolist()
    test_x=test_df[['day_from_jan_first','month', 'day']]
    test_x_fat = test_df[['day_from_jan_first','month', 'day']]
    test_y1=model_1.predict(test_x)
    test_x_fat['ConfirmedCases']=np.rint(test_y1)
    test_y2=model_2.predict(test_x_fat)
    test_res=pd.DataFrame(columns=submission.columns)
    test_res['ConfirmedCases']=np.rint(test_y1)
    test_res['ForecastId']=test_id
    test_res['Fatalities']=np.rint(test_y2)
    submission=submission.append(test_res)

C:\Users\PP\Anaconda3\lib\site-packages\ipykernel_launcher.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [129]:
submission.head()

,ForecastId,ConfirmedCases,Fatalities
0,15049,0.0,0.0
1,15051,0.0,0.0
2,15055,0.0,0.0
3,15057,0.0,0.0
4,15068,0.0,0.0


In [ ]:
submission.to_csv('submission.csv',index=False)